-------------
# Differential Side-Channel Attack on XMEGA AES traces
#### Unsupervised
Here we are using:

- the open source library scared https://gitlab.com/eshard/scared
- the open source side-channel library Lascar is also an equivalent alternative https://github.com/Ledger-Donjon/lascar
    
-------------



In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import estraces
import scared

In [ ]:
import numpy as np

In [ ]:
vhex = np.vectorize(hex)

In [ ]:
ths = estraces.read_ths_from_ets_file('../SideCA-Traces/AES_xmega_traces_set_NewAE.ets')
print(ths)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.rcParams['figure.figsize']=(16,4)

In [ ]:
plt.plot(ths.samples[0:1].T)
plt.title('AES traces')
plt.show()

In [ ]:
len(ths)

In [ ]:
ths.metadata_tags

In [ ]:
# Create a container for your ths
container = scared.Container(ths)

### Where to attack?

#### see slides

----
## Attack the first round with DPA

In [ ]:
attack_dpa = scared.DPAAttack(
        selection_function=scared.aes.selection_functions.encrypt.FirstSubBytes(),
        model=scared.Monobit(0),
        discriminant=scared.maxabs,
        convergence_step=len(ths)//5    
    )

In [ ]:
attack_dpa.run(container)

In [ ]:
found_key = attack_dpa.scores.argmax(0).squeeze()

In [ ]:
vhex = np.vectorize(hex)
print("the key found is: ", vhex(found_key))

#### Plot Results

In [ ]:
import aLibrary.attack_results_scared_dist as scared_attack_result

In [ ]:
figure_name_path = './attack_dpa_xmega_'

In [ ]:
attack_result = scared_attack_result.attack_results_scared_dist(attack_dpa, figure_name_path, file_name_path=None)

In [ ]:
attack_result.plot(0, ths.key[0], do_not_save_figure_bool=True)

### Print attack results

In [ ]:
attack_result.print_scores(ths.key[0])

------ 
## Plot Remaining Key Entropy and Traces Number 

#### Naive entropy estimation:
- worst cases
- best cases



In [ ]:
key_rank_lower, key_rank_est, key_rank_max = attack_result.remaining_entropy_naive(ths.key[0], do_not_save_figure_bool=True)

------ 
## Plot Remaining Key Entropy and Traces Number 

Improved estimation method in

- [GlowaczGPSS15] Cezary Glowacz, Vincent Grosso, Romain Poussier, Joachim Schüth, François-Xavier Standaert:
Simpler and More Efficient Rank Estimation for Side-Channel Security Assessment. FSE 2015: 117-129




In [ ]:
key_rank_lower, key_rank_est, key_rank_max = attack_result.remaining_entropy(ths.key[0], do_not_save_figure_bool=True)

----
## Attack the first round with CPA

In [ ]:
attack_cpa = scared.CPAAttack(
        selection_function=scared.aes.selection_functions.encrypt.FirstSubBytes(),
        model=scared.HammingWeight(),
        discriminant=scared.maxabs,
        convergence_step=len(ths)//5    
    )

In [ ]:
attack_cpa.run(container)

In [ ]:
found_key = attack_cpa.scores.argmax(0).squeeze()

#### Plot and Save Results

In [ ]:
figure_name_path = './attack_cpa_xmega_'

In [ ]:
attack_result = scared_attack_result.attack_results_scared_dist(attack_cpa, figure_name_path=figure_name_path, file_name_path=None)

In [ ]:
attack_result.plot(0, ths.key[0])

In [ ]:
plt.rcParams['figure.figsize']=(16,8)

plt.subplot(2,1,1)
plt.plot(ths.samples[0:1].T[0:200])
plt.title('AES traces')

plt.subplot(2,1,2)
plt.plot(attack_cpa.results[:,0,0:200].T)
plt.show()

#### Plot and Save ALL Bytes Results

In [ ]:
attack_result.plot(None, ths.key[0])

### Print attack results

In [ ]:
attack_result.print_scores(ths.key[0])

------ 
## Plot Remaining Key Entropy and Traces Number 

#### Naive entropy estimation:
- worst cases
- best cases



In [ ]:
key_rank_lower, key_rank_est, key_rank_max = attack_result.remaining_entropy_naive(ths.key[0])

------ 
## Plot Remaining Key Entropy and Traces Number 

Improved estimation method in

- [GlowaczGPSS15] Cezary Glowacz, Vincent Grosso, Romain Poussier, Joachim Schüth, François-Xavier Standaert:
Simpler and More Efficient Rank Estimation for Side-Channel Security Assessment. FSE 2015: 117-129




In [ ]:
key_rank_lower, key_rank_est, key_rank_max = attack_result.remaining_entropy(ths.key[0])

### Print attack best results

In [ ]:
attack_result.print_best_scores(5)

-----
## Attack the last round in delta with ANOVA

In [ ]:
attack_anova = scared.ANOVAAttack(
        selection_function=scared.aes.selection_functions.encrypt.FirstSubBytes(),
        model=scared.HammingWeight(),
        discriminant=scared.maxabs,
        convergence_step=len(ths)//5    
    )

In [ ]:
attack_anova.run(container)

#### Plot Results

In [ ]:
attack_result = scared_attack_result.attack_results_scared_dist(attack_anova)

In [ ]:
attack_result.print_scores(ths[0].key)

In [ ]:
key_rank_lower, key_rank_est, key_rank_max = attack_result.remaining_entropy(ths[0].key)

In [ ]:
attack_result.plot(0, ths[0].key)